In [1]:
import math
import time
import cv2
import cvzone
from ultralytics import YOLO

# ========== Configuration ==========
CONFIDENCE_THRESHOLD = 0.7
MODEL_PATH = "your trained model"
CLASS_NAMES = ["fake", "real"]
CAMERA_WIDTH, CAMERA_HEIGHT = 416, 416

# ========== Initialize Webcam ==========
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # Use 1 if external camera
cap.set(3, CAMERA_WIDTH)
cap.set(4, CAMERA_HEIGHT)

# ========== Load YOLOv8 Model ==========
model = YOLO(MODEL_PATH)

# ========== FPS Calculation ==========
prev_frame_time = 0

# ========== Main Loop ==========
while True:
    new_frame_time = time.time()
    success, frame = cap.read()
    if not success:
        print("❌ Failed to grab frame")
        break

    # Run inference
    results = model(frame, stream=True, verbose=False)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            w, h = x2 - x1, y2 - y1

            confidence = math.ceil(box.conf[0] * 100) / 100
            class_id = int(box.cls[0])

            if confidence > CONFIDENCE_THRESHOLD:
                label = CLASS_NAMES[class_id]
                color = (0, 255, 0) if label == "real" else (0, 0, 255)

                # Draw bounding box and label
                cvzone.cornerRect(frame, (x1, y1, w, h), colorC=color, colorR=color)
                cvzone.putTextRect(
                    frame,
                    f'{label.upper()} {int(confidence * 100)}%',
                    (max(0, x1), max(35, y1)),
                    scale=2,
                    thickness=4,
                    colorR=color,
                    colorB=color
                )

    # Show FPS
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    cv2.putText(frame, f'FPS: {int(fps)}', (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Display the result
    cv2.imshow("YOLOv8 Real-Time Detection", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ========== Cleanup ==========
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cvzone'